In [1]:
import pandas as pd
import os
import glob
import pandas as pd
from generate_overview import generate_summary
from utils.plotter import plot_donations_by_date, plot_data_points_by_category

# Get list of all CSV files in the overview directory
overview_files = glob.glob('./data/overview/overview_*.csv')
# Find most recent file based on modification time
latest_file = max(overview_files, key=os.path.getmtime)
# Read the most recent CSV file
df = pd.read_csv(latest_file)

# Filter entries before start of data collection -> must be refined. (17th offical launch)
df = df[df['start_time'] >= '2025-02-17']

# Filter out 99 (test case)
# Filter out explicit strings from Q2_age while keeping numeric values or empty values (removes testcases)
string_mask = (df['Q2_age'].astype(str).str.match(r'^[A-Za-z]+$')) & (df['Q2_age'].astype(str) != 'nan')
df = df[~string_mask]
df = df[df['Q2_age'] != '99']
n_started = df.shape[0]


n_finished = df['completed'].eq(True).sum()
df = df[pd.to_numeric(df['Q2_age'], errors='coerce').notna()]



/Users/lion.wedel/gitProjects/datadonation-wi-monitoring/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
len(df)

764

In [12]:
df_completed = df[df["completed"] == True]
len(df_completed)


764

In [13]:
df_consent = df_completed[df_completed["AngeseheneVideos_consent"] == True]
len(df_consent)


713

In [14]:
# First convert the end_time column to datetime
df_consent['end_time'] = pd.to_datetime(df_consent['end_time'])

# Now you can use the dt accessor
df_today = df_consent[df_consent["end_time"].dt.date == pd.to_datetime("2025-02-27").date()]

/var/folders/qn/fxb4jn2d09gbf70_jjtqy3dc0000gr/T/ipykernel_85903/706759970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consent['end_time'] = pd.to_datetime(df_consent['end_time'])


In [17]:
for i in df_today.columns:
    print(i)


participant_id
start_time
end_time
completed
extra_data
current_step
Q1_gender
Q2_age
Q3_education
Q4_location
Q5_first_vote
Q6_second_vote
Q7_polInt-0
Q7_polInt-1
time_submitted
handled
AngeseheneVideos_consent
AngeseheneVideos_status
AngeseheneVideos_n_datapoints
Likes_consent
Likes_status
Likes_n_datapoints
Suchen_consent
Suchen_status
Suchen_n_datapoints
Shares_consent
Shares_status
Shares_n_datapoints
Posts_consent
Posts_status
Posts_n_datapoints
Kommentare_consent
Kommentare_status
Kommentare_n_datapoints
FolgendeAccounts_consent
FolgendeAccounts_status
FolgendeAccounts_n_datapoints
GefolgteAccounts_consent
GefolgteAccounts_status
GefolgteAccounts_n_datapoints
BlockierteAccounts_consent
BlockierteAccounts_status
BlockierteAccounts_n_datapoints


In [19]:
df_check = df_today.loc[:,['participant_id',"AngeseheneVideos_status",'AngeseheneVideos_n_datapoints',
                'Likes_status','Likes_n_datapoints',
                'Suchen_status','Suchen_n_datapoints',
                'Shares_status','Shares_n_datapoints',
                'Posts_status','Posts_n_datapoints',
                'Kommentare_status','Kommentare_n_datapoints',
                'FolgendeAccounts_status','FolgendeAccounts_n_datapoints',
                'GefolgteAccounts_status','GefolgteAccounts_n_datapoints',
                'BlockierteAccounts_status','BlockierteAccounts_n_datapoints']]


In [21]:
df_check[df_check["AngeseheneVideos_n_datapoints"] == 0].describe()

,AngeseheneVideos_n_datapoints,Likes_n_datapoints,Suchen_n_datapoints,Shares_n_datapoints,Posts_n_datapoints,Kommentare_n_datapoints,FolgendeAccounts_n_datapoints,GefolgteAccounts_n_datapoints,BlockierteAccounts_n_datapoints
count,29.0,29.0,29.0,29.0,29.000000,29.000000,29.0,29.0,29.000000
mean,0.0,0.0,0.0,0.0,3.586207,213.965517,0.0,0.0,7.551724
std,0.0,0.0,0.0,0.0,12.835546,474.359755,0.0,0.0,31.739550
min,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
25%,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
50%,0.0,0.0,0.0,0.0,0.000000,10.000000,0.0,0.0,0.000000
75%,0.0,0.0,0.0,0.0,0.000000,50.000000,0.0,0.0,4.000000
max,0.0,0.0,0.0,0.0,60.000000,1646.000000,0.0,0.0,172.000000


In [6]:
df_failed = df_today[df_today["AngeseheneVideos_n_datapoints"] == 0]

In [7]:
df_today[df_today["AngeseheneVideos_n_datapoints"] > 0]

,participant_id,start_time,end_time,completed,extra_data,current_step,Q1_gender,Q2_age,Q3_education,Q4_location,...,Kommentare_n_datapoints,FolgendeAccounts_consent,FolgendeAccounts_status,FolgendeAccounts_n_datapoints,GefolgteAccounts_consent,GefolgteAccounts_status,GefolgteAccounts_n_datapoints,BlockierteAccounts_consent,BlockierteAccounts_status,BlockierteAccounts_n_datapoints
615,6GeJrvD80BaQhNBbP6ThpqRL,2025-02-26T16:21:26.731966+01:00,2025-02-26 16:23:07.887584+01:00,True,{'url_param': {}},3,0.0,18,6.0,1.0,...,50.0,True,success,166.0,True,success,3105.0,True,success,1.0
662,6fkxrzPW58udQan3iQuSxLTA,2025-02-26T22:55:32.375629+01:00,2025-02-26 23:12:32.071131+01:00,True,{'url_param': {}},3,0.0,23,6.0,9.0,...,100.0,True,success,13.0,True,success,270.0,True,success,26.0
764,7ey572gNxYgMnJdHak5nzYWi,2025-02-26T22:53:32.233295+01:00,2025-02-26 23:03:43.218283+01:00,True,{'url_param': {}},3,1.0,29,7.0,10.0,...,9.0,True,success,22.0,True,success,116.0,True,success,4.0
797,83WznQTrHFPP1NEgfPys0dOg,2025-02-26T22:34:42.160418+01:00,2025-02-26 22:51:26.099467+01:00,True,{'url_param': {}},3,0.0,26,4.0,1.0,...,18.0,True,success,21.0,True,success,483.0,True,success,3.0
939,9PaNFOIuqW8bzIUoKAhfEjRc,2025-02-26T23:04:54.351360+01:00,2025-02-26 23:19:11.575995+01:00,True,{'url_param': {}},3,0.0,20,6.0,0.0,...,29.0,True,success,1.0,True,success,30.0,True,success,0.0
1006,A2ZJ0RkmiVGcGStmCChMW8Ds,2025-02-26T19:24:15.374681+01:00,2025-02-26 23:27:54.014036+01:00,True,{'url_param': {}},3,0.0,19,6.0,15.0,...,653.0,True,success,605.0,True,success,6336.0,True,success,1.0
1036,AHrNOzjNC248JkGa7D4W92yt,2025-02-26T23:43:33.277363+01:00,2025-02-26 23:52:21.388765+01:00,True,{'url_param': {}},3,0.0,33,6.0,0.0,...,3.0,True,success,7.0,True,success,327.0,True,success,0.0
1046,AMbmQLJI9ir423zAupfu4JP8,2025-02-26T16:20:58.645714+01:00,2025-02-26 22:20:53.381625+01:00,True,{'url_param': {}},3,0.0,30,7.0,6.0,...,1136.0,True,success,2716.0,True,success,842.0,True,success,25.0
1155,BTEFUmbieOzUOBCXDvKLHdhL,2025-02-26T21:54:16.793402+01:00,2025-02-26 22:57:05.982781+01:00,True,{'url_param': {}},3,0.0,25,8.0,8.0,...,50.0,True,success,3.0,True,success,141.0,True,success,5.0
1173,BhjMBuN8zBWY569UOfUhrDHF,2025-02-26T14:09:49.729061+01:00,2025-02-26 23:30:34.718787+01:00,True,{'url_param': {}},3,0.0,24,7.0,0.0,...,1133.0,True,success,20.0,True,success,142.0,True,success,4.0


In [8]:
sus_ids = df_failed.participant_id

In [9]:
for id in sus_ids:
    ddp = 

SyntaxError: invalid syntax (591377912.py, line 2)

In [10]:
df_withWH = df_consent[df_consent["AngeseheneVideos_n_datapoints"] > 0]
len(df_withWH)

569

In [11]:

df_usable = df_usable[df_usable["AngeseheneVideos_n_datapoints"] > 0]
df_usable.to_csv('./data/overview/usable_overview.csv', index=False)


Age statistics:
count    320.00
mean      28.37
std        9.29
min       13.00
50%       26.00
max       69.00
Name: Q2_age, dtype: float64


In [13]:
# Use this instead:
pd.concat([
    df_usable.loc[:,['Q1_gender','Q2_age','Q3_education','Q7_polInt-0']].describe(percentiles=[0.5]).round(2), 
    df_usable.loc[:,['Q2_age']].dropna().astype(int).describe(percentiles=[0.5]).round(2)
], axis=1)


,Q1_gender,Q2_age,Q3_education,Q7_polInt-0,Q2_age
count,320.00,320.00,320.00,264.00,320.00
mean,0.48,28.37,5.97,7.33,28.37
std,0.55,9.29,2.22,1.48,9.29
min,0.00,13.00,0.00,2.00,13.00
50%,0.00,26.00,6.00,7.00,26.00
max,3.00,69.00,10.00,9.00,69.00
